# TouchTerrain standalone in a jupyter notebook
Chris Harding, Mar.1, 2019

- this is the jupyter notebook version of `TouchTerrain_standalone.py`
- this notebook needs to be run in Python __2.7__
- I assume you've installed anaconda or miniconda and have the standard 3.party packages already install (e.g. numpy, pillow (aka PIL), etc.)
- the following additional packages are required, either use (ana)conda or pip to install:
    - gdal (On windows, if conda gives you trouble, get the whl file from [here](https://www.lfd.uci.edu/~gohlke/pythonlibs/) and install via `pip install <whl file>`
    - vectors: install via `pip install vectors`
    - earthengine-api (ONLY if you want to use Google Earth Engine's (GEE) online DEM data, NOT needed for converting local DEM raster files (geotiffs) 


- if you're never going to use Google Earth Engine's (GEE) online DEM data skip the next section and go to __Running TouchTerrain__

### Using Google Earth Engine online DEM rasters 

- You don't need this if you only want to import terrain from locally stored raster files (geotiffs)!
- TouchTerrain can use DEM data from Google Earth Engine (given the corners of the area), but you need to first request a developer account and set up an authentication file
- (This dev account is different from your standard Google (Gmail, etc.) account!)
- Getting the account is free and entitles you to a modest number of requests (4 per seconds), which are also free. To request got to https://signup.earthengine.google.com/, you'll get and email with a file. 
- refer to the part __Setting Up Authentication Credentials__ https://developers.google.com/earth-engine/python_install_manual (ignore the stuff above it, as you should already have installed all the needed packages when you installed the earthengine-api package ...)

In [ ]:
###

In [ ]:
# comment out (remove the #) the next line to see if you could use ee (Earth Engine)
#import ee; ee.Initialize()

if you get: 

`Please authorize access to your Earth Engine account by running earthengine authenticate in your command line, and then retry.`

comment out and run the cell below, the ! means it will run `earthengine authenticate` inside a OS shell (commandline). 

- This will asked to to sign in to your Google Account and give you a very long authentication token. 
- Paste that token in, to your commandline
- if correct it will create a folder `.config` in your home folder and create a authentication file (in a earthengine folder)
- with this file in place go back to the cell above (`import ee;ee.Initialize()`) and run it
- this time, it should work and you now have access to the Google Earth engine API and it's terrain data (DEM sources)

In [ ]:
#!earthengine authenticate

## Running TouchTerrain

Click on the cell below and hit Shift-Enter to run it (won't show anything, just does some path setup stuff)   

In [ ]:
# RUN THIS CELL TO START

# just some setup stuff - nothing to see here
import os, sys
from os.path import abspath
from pprint import pprint
from glob import glob
this_folder = abspath(os.getcwd())
last = this_folder.rindex(os.sep)
common_folder = this_folder[:last] + os.sep + os.sep + "common"
sys.path.append(common_folder) # add common folder to sys.path
#print sys.path

Put your settings into the dictionary below and  hit Shift-Enter

- for more info on the settings, look at the ReadMe on https://github.com/ChHarding/TouchTerrain_for_CAGEO
- note, however, that the settings given below are in Python syntax, whereas the ReadMe describes the JSON syntax used in the config file
- both are very similar except for None and True/False
- __Python__ vs JSON:
    - `None  null`
    - `True  true`
    - `False false`

In [ ]:
args = {
    # DEM/Area to print
    
    # A: use local DEM raster (geotiff)
    #"importedDEM": "pyramid_wide.tif",  # put file in same folder as this notebook file!
    
    # B: use area and a DEM online source via EarthEngine
    "importedDEM": None,
    "DEM_name": "USGS/NED",   # DEM source
    "bllat": 44.50185267072875,   # bottom left corner lat
    "bllon": -108.25427910156247, # bottom left corner long
    "trlat": 44.69741706507476,   # top left corner lat
    "trlon": -107.97962089843747, # top left corner long
    
    # 3D print parameters
    "tilewidth": 50,  # width of each tile in mm, (tile height will be auto calculated)
    "printres": 0.4,  # resolution (horizontal) of 3D printer (= size of one pixel) in mm, 
                      # should be your nozzle size or just a bit less! 
                      # Using something like 0.01 will NOT print out a super detailed version 
                      # as you slicer will remove such fine details anyway! Instead, you'll
                      # just wait a long time and get a super large STL file!
                      # If you want the original resolution of the DEM, use -1
    "ntilesx": 1, # number of tiles in x  
    "ntilesy": 1, # number of tiles in y    

    "basethick": 0.5,   # thickness (in mm) of printed base
    "zscale": 3,      # elevation (vertical) scaling
    "fileformat": "STLa",  # format of 3D model files: "obj" wavefront obj (ascii),
                           #   "STLa" ascii STL or "STLb" binary STL
                           #   to export just the(untiled)raster (no mesh) use "GeoTiff" 
    "zip_file_name": "myterrain",   # base name of zipfile, .zip will be added

    
    # Expert settings
    "tile_centered": False, # True-> all tiles are centered around 0/0, False, all tiles "fit together"
    "CPU_cores_to_use" : 0, # 0 means all cores, None => don't use multiprocessing
                            # multi-core will be much faster for more than 1 tile 
    "max_cells_for_memory_only" : 9999999999999, # if number of raster cells is bigger than this, use temp_files instead of memory
                            #   set this very high to force use of memory and lower it if you run out of memory
    "no_bottom": False,    # omit bottom triangles? Most slicers still work and it makes smaller files
    "bottom_image": None, # 1 band greyscale image used for bottom relief
    "ignore_leq": None,   # set all values <= this to NaN so they don't print
    "unprojected": False, # don't project to UTM (for EE rasters only)
    "only" : None,        # if not None: list with x and y tile index (1 based) of the only tile to process
                          #   e.g. [1,1] will only process the tile in upper left corner, [2,1] the tile right to it, etc.
}

########################################################

# if we want to work on a local rasterg et the full pathname to it
if args["importedDEM"] != None: 
    args["importedDEM"]= abspath(args["importedDEM"]) 
#pprint(args)

In [ ]:
import TouchTerrainEarthEngine as TouchTerrain

# Process the data

# This may take some time! You'll see In[*] and some log messages (will also be in the logfile inside the zip)
# You may see some red stuff with 10%, etc. - don't worry, that's normal

totalsize, full_zip_file_name = TouchTerrain.get_zipped_tiles(**args) # all args are in a dict
print "\nDONE!\n\nCreated zip file", full_zip_file_name,  "%.2f" % totalsize, "Mb"

# your zip file will be inside the tmp folder which is inside the same folder your notebook file is in

In [ ]:
# If you want to unzip the zip file, run this cell

# (You will need to do this before using k3d for visualiztion)

import os.path
folder, file = os.path.splitext(full_zip_file_name) # get folder of zip file

# unzip the zipfile into the folder it's already in
import zipfile
zip_ref = zipfile.ZipFile(full_zip_file_name, 'r')
zip_ref.extractall(folder)
zip_ref.close()
print "unzipped files from", full_zip_file_name, "into", folder
print "folder contains these files:"
for f in glob(folder + os.sep + "*.*"): print " ", f

## Visualize the STL file(s)

- If you want to visualize your model(s) install k3d (`pip install k3d`) and run the cell below
- __Works only with ascii STLs (stla)!__

In [ ]:
import k3d

# get all stl files in that folder
mesh_files = glob(folder + os.sep + "*.stl")
#print "in folder", folder, "using", mesh_files

plot = k3d.plot()

from random import randint
for m in mesh_files:
    print "adding to viewer:", m
    col = (randint(0,255) << 16) + (randint(0,255) << 8) + randint(0,255) # random rgb color as hex
    buf = open(m, 'r').read()
    plot += k3d.stl(buf, color=col)
plot.display()

In [ ]:
# CH test settings
args = {

            "DEM_name": 'USGS/NED',# DEM_name:    name of DEM source used in Google Earth Engine
                                   # for all valid sources, see DEM_sources in TouchTerrainEarthEngine.py
            "trlat": 44.62708958762245,        # lat/lon of top right corner
            "trlon": -108.09017082519529,
            "bllat": 44.59776164299389,        # lat/lon of bottom left corner
            "bllon": -108.19179436035154,
            
            # salt lake
            "trlat": 41.3889181444,
            "trlon": -112.336582824,
            "bllat": 41.1951151099,
            "bllon": -112.622227355,
    
    
            "importedDEM": None,
            "importedDEM": "offshore.tif",
            "importedDEM": "test_terrain.tif", # if not None, the raster file to use as DEM instead of using GEE (null in JSON)
            "printres": -1,  # resolution (horizontal) of 3D printer (= size of one pixel) in mm
            "ntilesx": 5,      # number of tiles in x and y
            "ntilesy": 5,
            "tilewidth": 100, # width of each tile in mm (<- !!!!!), tile height is calculated
            "basethick": 0, # thickness (in mm) of printed base
            "zscale": 1.0,      # elevation (vertical) scaling
            "fileformat": "STLa",  # format of 3D model files: "obj" wavefront obj (ascii),"STLa" ascii STL or "STLb" binary STL
            "tile_centered": False, # True-> all tiles are centered around 0/0, False, all tiles "fit together"
            "zip_file_name": "only_test",   # base name of zipfile, .zip will be added
            "CPU_cores_to_use" : None,  # 0 means all cores, None (null in JSON!) => don't use multiprocessing
            "max_cells_for_memory_only" : 0,#100000 * 100000, # if raster is bigger, use temp_files instead of memory
            "no_bottom": True, # omit bottom triangles?
            "bottom_image": None,  # 1 band greyscale image used for bottom relief
            "ignore_leq": None, # set values <= this to NaN,
            "unprojected": False, # don't project to UTM, only useful when using GEE for DEM rasters
            "only": [2,1], # list of tile index [x,y] with is the only tile to be processed. None means process all tiles (index is 1 base
}
